Firebase URL: https://console.firebase.google.com/u/0/project/morphine-64cdd/database/morphine-64cdd-default-rtdb/data/~2F 

In [ ]:
import firebase_admin
from firebase_admin import db
from datetime import datetime
import time
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Connect to Firebase

In [ ]:
cred_obj = firebase_admin.credentials.Certificate("Morphine2.json")
default_app = firebase_admin.initialize_app(cred_obj, {
  'databaseURL':'https://morphine-64cdd-default-rtdb.asia-southeast1.firebasedatabase.app/'
  })
users_data = db.reference("/Users Data/Token UID:XvIeVwC7M0QN0qW15FNYO2e5BJ93")

In [ ]:
data = users_data.get()

## Process ESP32 Output

In [ ]:
# dataframe to store esp32 data points
users_data_df = pd.DataFrame(
    columns = ['accounter', 'LoopSpeedArr', 'UploadSpeedArr', 'set_index', 'Ax', 'Ay', 'Az', 'gx', 'gy', 'gz',
               'temp', 'timingForThisSet', 'timeDifference'])

In [ ]:
# keywords to extract for each set
keywords = ['Ax: ', 'Ay: ', 'Az: ', 'gx: ', 'gy: ', 'gz: ', 'temp: ', 'timing for this set: ']

# length of each keyword
length_keywords = [len(x) for x in keywords]
#print(length_keywords)

In [ ]:
# function to process one datapoint
def process_one_set_of_datapoint(output_set):
    indexes = [output_set.find(keyword) for keyword in keywords]
    df_row = []
    curr_data_index = int(output_set[:indexes[0]].strip())
    df_row.append(curr_data_index) # append in the index of the new input
    for i, index in enumerate(indexes):
        if i == len(indexes) - 1:
            x = float(output_set[index+length_keywords[i]:].strip())
            df_row.append(x)
        else:
            x = float(output_set[index+length_keywords[i]: indexes[i+1]].strip())
            df_row.append(x)
    return df_row

In [ ]:
# function to extract every variables for every data input (with 20 sets/datapoints) read in
def user_data_process_esp32_output(esp32_output, timeDifference):
    global users_data_df
    accounter = esp32_output['Accounter']
    LoopSpeedArr = esp32_output['LoopSpeedArr'][0]
    UploadSpeedArr = esp32_output['UploadSpeedArr'][0]
    #gps_info = esp32_output['GPS'] # removed, relocate under split circuit
    datapoints = esp32_output['Datapoints'][0]
    output_sets = datapoints.split('Set: ')[1:]
    for output_set in output_sets:
        data = [accounter, LoopSpeedArr,UploadSpeedArr]
        datapoint = process_one_set_of_datapoint(output_set)
        data.extend(datapoint)
        data.append(timeDifference)
        new_df = pd.DataFrame([data], 
                              columns = ['accounter', 'LoopSpeedArr', 'UploadSpeedArr', 'set_index', 'Ax', 'Ay', 'Az', 'gx', 'gy', 'gz', 'temp', 'timingForThisSet', 'timeDifference'])
        users_data_df = pd.concat([users_data_df, new_df], ignore_index = True)

## Process Split Circuit Data

### GPS Data

In [ ]:
gps_keywords_list = ['Latitude: ', 'Longitude: ', 'Altitude: ', 'Satellite-in-view: ', 'timing for this set: ']
gps_keywords_length = [len(keyword) for keyword in gps_keywords_list]
#gps_keywords_length

In [ ]:
# feed in gps_datapoints = gps_data['GPS Datapoints']
def process_gps_datapoints(gps_datapoints):
    gps_datapoints = gps_datapoints[0]
    indices = [gps_datapoints.find(keyword) for keyword in gps_keywords_list]
    # Latitude, Longtiude, Altitude, Satellite-in-view, timing for this set
    gps_datapoints_list = []
    for i, index in enumerate(indices):
        if i == len(indices) - 1:
            gps_datapoints_list.append(gps_datapoints[index + gps_keywords_length[i]:].strip())
        else:
            gps_datapoints_list.append(gps_datapoints[index + gps_keywords_length[i]:indices[i+1]].strip())
    return gps_datapoints_list

In [ ]:
# feed in data['Split Circuit']['GPS']
def process_gps(gps):
    processed_gps_data = []
    gps_accounter = gps['GPS Accounter']
    gps_datapoints = gps['GPS Datapoints']
    gps_loopSpeedArr = gps['GPS LoopSpeedArr'][0]
    gps_uploadSpeedArr = gps['GPS UploadSpeedArr'][0]
    
    processed_gps_datapoints = process_gps_datapoints(gps_datapoints)
    
    processed_gps_data.append(gps_accounter)
    processed_gps_data.extend(processed_gps_datapoints)
    processed_gps_data.extend([gps_loopSpeedArr, gps_uploadSpeedArr])
    return processed_gps_data

### GPS Button
split_circuit_data['GPS Button']

### MPU6050

In [ ]:
mpu6050_df = pd.DataFrame(columns = ['accounter', 'LoopSpeedArr', 'UploadSpeedArr', 'set_index', 'Ax', 'Ay', 'Az', 'gx', 'gy', 'gz', 'temp', 'timingForThisSet', 'timeDifference'])
#display(mpu6050_df)

In [ ]:
# feed in mpu6050_output = split_circuit_data['MPU6050']
def process_mpu6050(mpu6050, timeDifference):
    global mpu6050_df
    accounter = mpu6050['MPU6050 Accounter']
    mpu6050_datapoints = mpu6050['MPU6050 Datapoints'][0]
    mpu6050_loopSpeedArr = mpu6050['MPU6050 LoopSpeedArr'][0]
    mpu6050_uploadSpeedArr = mpu6050['MPU6050 UploadSpeedArr'][0]
    mpu6050_output_sets = mpu6050_datapoints.split('Set: ')[1:]
    
    for output in mpu6050_output_sets:
        data = [accounter, mpu6050_loopSpeedArr,mpu6050_uploadSpeedArr]
        datapoint = process_one_set_of_datapoint(output)
        data.extend(datapoint)
        data.append(timeDifference)
        new_df = pd.DataFrame([data], 
                              columns = ['accounter', 'LoopSpeedArr', 'UploadSpeedArr', 'set_index', 'Ax', 'Ay', 'Az', 'gx', 'gy', 'gz', 'temp', 'timingForThisSet', 'timeDifference'])
        mpu6050_df = pd.concat([mpu6050_df, new_df], ignore_index = True)

In [ ]:
# overall function to read split circuit, feed in split_circuit_data = data['Split Circuit']
def process_split_ciruit_data(split_circuit_data):
    keys = split_circuit_data.keys()
    for key in keys:
        if key == 'GPS':
            gps_data = split_circuit_data['GPS']
            print(process_gps(gps_data))
        elif key == 'GPS Button':
            print(split_circuit_data['GPS Button'])
        elif key == 'MPU6050':
            mpu6050_output = split_circuit_data['MPU6050']
            process_mpu6050(mpu6050_output, None)
            print('Loaded MPU6050 Data into mpu6050_df')

In [ ]:
# need to store the gps data

In [ ]:
# need to store everything tgt : eps32 output data and split circuit data

## Try Reading Data

overall functions:
1. user_data_process_esp32_output(esp32_output, timeDifference)
2. process_split_ciruit_data(split_circuit_data)

In [ ]:
esp32_output = users_data.get()
split_circuit_data = esp32_output['Split Circuit']

In [ ]:
user_data_process_esp32_output(esp32_output, None)

In [ ]:
users_data_df

In [ ]:
process_split_ciruit_data(split_circuit_data)

In [ ]:
mpu6050_df

In [ ]:
def read_data():
    data = users_data.get()
    split_circuit_data = data['Split Circuit']
    user_data_process_esp32_output(data, None)
    process_split_ciruit_data(split_circuit_data)

In [ ]:
read_data()